# Importing libraries

In [1]:
import pandas as pd
import os
import csv
import datetime
import time
from pytz import timezone
import tweepy
import json

# Loading the credentials

In [2]:
# Load the Twitter API credentials from the config file
with open('config.json', 'r') as f:
    config = json.load(f)
    consumer_key = config['consumer_key']
    consumer_secret = config['consumer_secret']
    access_token = config['access_token']
    access_token_secret = config['access_token_secret']
    
# Verify the Twitter API credentials
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
try:
    api = tweepy.API(auth, wait_on_rate_limit=True)
    user = api.verify_credentials()
    print("Twitter API connection successful.")
except tweepy.error.TweepError as e:
    print("Error: Failed to verify Twitter API credentials.")
    print(e)

Twitter API connection successful.


# Disabling warnings

In [3]:
import warnings
warnings.filterwarnings("ignore")

# Downloading twitter data

In [ ]:
# Define the topic and date range
topic = 'Ireland'
end_date = datetime.datetime(2023, 5, 21, tzinfo=timezone('Europe/London'))
start_date = datetime.datetime(2022, 5, 21, tzinfo=timezone('Europe/London'))

# Fetch tweets on the specified topic
try:
    tweets = []
    for tweet in tweepy.Cursor(api.search_tweets, q=topic, lang='en', tweet_mode='extended').items(1000):
        if start_date <= tweet.created_at <= end_date:
            tweets.append(tweet.full_text)
    print("Tweets downloaded successfully.")
except tweepy.TweepyException as e:
    if e.api_code == 88:
        # Rate limit reached, wait for the specified duration
        wait_time = int(e.response.headers['Retry-After'])
        print("Rate limit reached. Sleeping for:", wait_time, "seconds.")
        time.sleep(wait_time)
    print("Error: Failed to download tweets.")
    print(e)

# Save the tweets to a CSV file
csv_filename = 'tweets.csv'
try:
    with open(csv_filename, 'w', encoding='utf-8', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Tweet'])
        for tweet in tweet:
            writer.writerow([tweet])
    print("Tweets saved to", csv_filename)
except IOError:
    print("Error: Failed to save tweets to CSV file.")